In [ ]:
import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
refuge_islands_path = os.path.join(base_dir, "osm_refuge_islands.gdb")
unassociated_refuge_islands_path = os.path.join(base_dir, "unassociated_refuge_islands.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)
refuge_islands = gpd.read_file(refuge_islands_path)

# Convert CRS_LEG from string to list
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reproject refuge islands to match roads CRS
refuge_islands = refuge_islands.to_crs(roads.crs)

# Add 'ID' column to refuge islands and assign sequential values
refuge_islands['ID'] = range(1, len(refuge_islands) + 1)

# Buffer refuge islands by 5 meters
refuge_islands['geometry'] = refuge_islands.geometry.buffer(5)

# Identify road segments that intersect with refuge islands
refuge_road_segments = roads[roads.geometry.intersects(refuge_islands.unary_union)]
refuge_road_ids = set(refuge_road_segments['unique_id'])

# Create a dictionary to store the nearest junction for each refuge island
junctions = crossings[['JUNC_ID', 'geometry']].drop_duplicates()  # Get unique junction points
island_to_junction = {}

for _, island in refuge_islands.iterrows():
    island_geom = island.geometry
    distances = junctions.geometry.distance(island_geom)
    nearest_junction_idx = distances.idxmin()
    nearest_junction_id = junctions.loc[nearest_junction_idx, 'JUNC_ID']
    island_to_junction[island['ID']] = nearest_junction_id

# Assign CRS_ISLAND only at crossings near the nearest junction
crossings['CRS_ISLAND'] = 0                 # Default to 0
for idx, row in crossings.iterrows():
    crs_leg = row['CRS_LEG']

    if isinstance(crs_leg, list):
        has_refuge_island = any(seg in refuge_road_ids for seg in crs_leg)

        # Ensure that the crossing is at the nearest junction for the island
        if has_refuge_island and row['JUNC_ID'] in island_to_junction.values():
            crossings.at[idx, 'CRS_ISLAND'] = 1  # Assign 1 only at nearest junction

# Identify unassociated refuge islands (those not linked to any road segment)
unassociated_refuge_islands = refuge_islands[~refuge_islands.intersects(refuge_road_segments.unary_union)]

# Save unassociated refuge islands to a new shapefile
unassociated_refuge_islands.to_file(unassociated_refuge_islands_path, driver='ESRI Shapefile')

# Save the updated crossings shapefile``
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print("Updated crossings.shp with CRS_ISLAND attribute for refuge islands.")
print(f"There are {len(unassociated_refuge_islands)} unassociated refuge islands.")

Updated crossings.shp with CRS_ISLAND attribute for refuge islands.
There are 70 unassociated refuge islands.


In [2]:
crossings['CRS_ISLAND'].value_counts()

CRS_ISLAND
0    41696
1      257
Name: count, dtype: int64

In [3]:
unc_island = crossings[(crossings['CRS_ISLAND'] == 1) & 
          (crossings['CONTROL'].isna() | (crossings['CONTROL'] == ""))]

unc_island_path = os.path.join(base_dir, "unc_island.shp")
unc_island.to_file(unc_island_path, driver='ESRI Shapefile')

print("Filtered crossings saved to 'unc_island.shp'")

Filtered crossings saved to 'unc_island.shp'


In [4]:
# Filter the crossings DataFrame to include only rows where CRS_ISLAND is 1
filtered_crossings = crossings[crossings['CRS_ISLAND'] == 1]

# Get the value counts for each category in the CONTROL column
control_value_counts = filtered_crossings['CONTROL'].value_counts(dropna=False)

print(control_value_counts)

CONTROL
signal          135
None             93
implied_stop     23
stop              6
Name: count, dtype: int64
